In [93]:
class SettingsHeft:
  def __init__(self, totalTask, totalEdge):
    self.totalTask = totalTask
    self.totalEdge = totalEdge

In [94]:
class Task:
  def __init__(self, id):
    self.id = id
    self.processorId = None
    self.rank = None
    self.compCost = []
    self.avgCompCost = None
    self.predEdges = []
    self.duration = {'start': None, 'end': None}

In [95]:
class Processor:
  def __init__(self, id):
    self.id = id
    self.taskList = []

In [96]:
class Edge:
  def __init__(self, predecessorTask, successorTask, data):
    self.predecessorTask = predecessorTask # id
    self.successorTask = successorTask # id
    self.data = data
    self.avgCommCost = None

In [97]:
import operator
import numpy as np
from random import randrange

# Settings HETF
settingsHeft = SettingsHeft(3, 3)

# Set of Edges
E = [Edge(0, 1, 8), Edge(0, 2, 7), Edge(1, 2, 4)]

# Set of tasks
V = [Task(0), Task(1), Task(2)]

# Graph of tasks link `v`x`v`
G = [[-1, 8, 7], [-1, -1, 4], [-1, -1, -1]] # -1 na diagonal pois é o mesmo processador

# Set of processors
Q = []

# Processors transfer rates `q`x`q`
B = [[0, 1], [2, 0]] # 0 na diagonal pois é o mesmo processador

# Communication startup costs
L = [1, 1, 1]

# Computation cost matrix `v`x`q`
W = [[2, 3], [1, 1], [1, 2]] # deixar apenas dentro de Task ou criar o W?

# Upward rank `v`
rankU = []

class HEFT:
  def __init__(self):
    self.numTasks = 0 # 3 # Contando a entry e a exit task
    self.numEdges = 0 # 3
    self.numProcessors = 0 # 2
    self.tasks = [] # [Task(0), Task(1), Task(2)]
    self.edges = [] # [Edge(0, 1, 0), Edge(0, 2, 0), Edge(1, 2, 0)]
    self.processors = [] # [Processor(0), Processor(1)]
    self.processorTransferRates = [[0, 1, 1], [1, 0, 1], [1, 1, 0]] # [[0, 1], [2, 0]] # Barramento heterogêneo?
    self.commStartupCosts = [1, 1] # Precisa??
    # self.compCosts = [[2, 3], [1, 1], [1, 2]] # Precisa??
    # self.rankU = [0] * self.numTasks
    # self.taskLinks = [[-1, 0, 0], [-1, -1, -1], [-1, -1, -1]] # Precida??

  def calculateAvgCompCost(self):
    for task in self.tasks:
      task.avgCompCost = np.average(task.compCost)
      print('avg', task.avgCompCost)

  def calculateAvgTransferRates(self):
    numProcessors = self.numProcessors
    numTransfers = (numProcessors * numProcessors) - numProcessors

    totalTransferRate = 0
    for i in range(numProcessors):
      for j in range(numProcessors):
        # if (i != j): Se manter o 0 na diagonal, não precisa desse IF
        totalTransferRate += self.processorTransferRates[i][j]

    return totalTransferRate / numTransfers

  def getEST(self, task, processor):
    est = 0
    lenTaskListProcessor = len(processor.taskList)

    for predEdge in task.predEdges:
      commCost = 0
      predTask = self.tasks[predEdge.predecessorTask] # error
      if predTask.processorId != processor.id:
        commCost = predEdge.data / self.processorTransferRates[predTask.processorId][processor.id] # Não esta sendo levado em consideração o communication statup cost

      if task.id == 9:
        print('preds', predEdge.predecessorTask, predTask.id, predTask.duration['end'], commCost) # remover
      est = max(est, predTask.duration['end'] + commCost)
    # for pred in self.tasks:
    #   if self.taskLinks[pred.id][task.id] != -1:  # if pred also done on processor, no communication cost
    #     c = self.taskLinks[pred.id][task.id] if pred.processorId != processor.id else 0 #TODO mudar esse communication cost para igual o arquivo
    #     est = max(est, pred.duration['end'] + c) # max (AFT(nm) + cm,i)
    freeTimes = []
    if lenTaskListProcessor == 0:       # no task has yet been assigned to processor
      freeTimes.append([0, float('inf')])
    else:
      for i in range(lenTaskListProcessor):
        if i == 0:
          if processor.taskList[i].duration['start'] != 0:    # if processor is not busy from time 0
            freeTimes.append([0, processor.taskList[i].duration['start']])
        else:
          freeTimes.append([processor.taskList[i-1].duration['end'], processor.taskList[i].duration['start']])
      freeTimes.append([processor.taskList[-1].duration['end'], float('inf')])
    for slot in freeTimes:     # free_times is already sorted based on avaialbe start times
      if est < slot[0] and slot[0] + task.compCost[processor.id] <= slot[1]:
        return slot[0]
      if est >= slot[0] and est + task.compCost[processor.id] <= slot[1]:
        return est

  def getEFT(self, task, processor, est):
    return task.compCost[processor.id] + est #self.getEST(task, processor)

  def calculateAvgCommCost(self):
    avgProcTransfer = self.calculateAvgTransferRates()
    avgCommStartup = np.average(self.commStartupCosts)
    print('avgtransfer', avgProcTransfer, avgCommStartup)

    for edge in self.edges:
      # edge.avgCommCost = avgCommStartup + edge.data / avgProcTransfer
      edge.avgCommCost = edge.data / avgProcTransfer # Não esta sendo levado em consideração o communication statup cost
      print('avg', edge.avgCommCost)

  def recursiveRankUpward(self, task):
    successorsMaxList = []
    # não precisa fazer esse loop se souber que é task de saida
    for edge in self.edges:
      if (edge.predecessorTask == task.id):
        succRankUpward = self.recursiveRankUpward(self.tasks[edge.successorTask])
        print(task.id, edge.avgCommCost, succRankUpward)
        successorsMaxList.append(edge.avgCommCost + succRankUpward)

    currentRankUpward = (task.avgCompCost if len(successorsMaxList) == 0
    else task.avgCompCost + np.amax(successorsMaxList))

    task.rank = currentRankUpward
    return currentRankUpward

  def calculateRankUpward(self):
    # assumindo que o primeiro vertice é a unica task de entrada
    self.recursiveRankUpward(self.tasks[0])
    self.rankUTasks = self.tasks[:]
    self.rankUTasks.sort(key = lambda x: x.rank, reverse=True)
    

  def scheduleTasks(self):
    firstTask = self.rankUTasks[0]
    processorId, compCost = min(enumerate(firstTask.compCost), key=operator.itemgetter(1))
    print("Task: ", firstTask.id, ", Proc: ", processorId, " -> EST: ", 0, 'EFT', w)
    firstTask.duration['start'] = 0
    firstTask.duration['end'] = compCost
    firstTask.processorId = processorId
    self.processors[processorId].taskList.append(firstTask)
  
    for task in self.rankUTasks[1:]:
      estBest = 0
      eftBest = float("inf")
      # task.duration['end'] = float("inf")
      for processor in self.processors:
        est = self.getEST(task, processor)
        eft = self.getEFT(task, processor, est)
        print("Task: ", task.id, ", Proc: ", processor.id, " -> EST: ", est, 'EFT', eft)
        if (eft < eftBest):
          estBest = est
          eftBest = eft
          bestProcessorId = processor.id
      task.duration['start'] = estBest
      task.duration['end'] = eftBest
      task.processorId = bestProcessorId
      self.processors[bestProcessorId].taskList.append(task)
      self.processors[bestProcessorId].taskList.sort(key = lambda x: x.duration['start'])

      # print(task.id, processor.id, self.getEFT(task, processor))
    
  def createProcessors(self, numProcessors):
    self.numProcessors = numProcessors
    for i in range(numProcessors):
      self.processors.append(Processor(i))

# calculateAvgCompCost(W, V)

# calculateAvgCommCost(L, B, E)

# rankU = [0] * settingsHeft.totalTask
# calculateRankUpward(V[0], V, E, rankU)
# print(rankU)

# def HEFT(task, processor):
#   task

In [98]:
class DAG:
  def __init__(self):
    self.colSize = 11

  def writeFormatCol(self, data):
    colContent = str(data).rjust(self.colSize)
    self.wf.write(colContent)

  def readFormatCol(self, line, indexCol):
    dataCol = line[ self.colSize*indexCol : self.colSize*(indexCol+1) ]
    print(dataCol, indexCol)
    return int(dataCol.strip())

  def isEntryOrExitTask(self, taskId, numTask):
    entryTaskId = 0 
    exitTaskId = numTask + 1
    return taskId == entryTaskId or taskId == exitTaskId
    

  # Basic STG Columns 
  # Task Id | Computation Cost | Num Predecessors | [ Predecessors Id list ]

  # Communication cost and heterogeneous processors (CCHP)
  # CCHP STG Columns 
  # Task Id | [ Computation Cost list ](length = numProc) | Num Predecessors | [ (Predecessors Id | Communication data) list ]
  def createCCHP(self, readDirfile, writeDirfile, numProcessors):
    self.rf = open(readDirfile, 'r')
    self.wf = open(writeDirfile, 'w')
    colSize = 11
  
    numTasks = int(self.rf.readline().strip())
    self.writeFormatCol(numTasks)
    self.writeFormatCol(numProcessors)
    self.wf.write("\n")

    for _ in range(numTasks+2): # +2 por causa da entry task e a exit task
      line = self.rf.readline()

      taskId = self.readFormatCol(line, 0)
      print(taskId)
      compCost = self.readFormatCol(line, 1)
      print(compCost)
      numPred = self.readFormatCol(line, 2)
      print(numPred,'\n')

      self.writeFormatCol(taskId)
      for _ in range(numProcessors):
        compCostProcessor = (randrange(compCost, compCost + 3) 
        if not self.isEntryOrExitTask(taskId, numTasks)
        else 0)
        
        self.writeFormatCol(compCostProcessor)
      
      self.writeFormatCol(numPred)
      for i in range(numPred):
        predTaskId = self.readFormatCol(line, i+3)
        
        commData = (randrange(1, 6) # random range 1 to 5
        if not (self.isEntryOrExitTask(taskId, numTasks) or self.isEntryOrExitTask(predTaskId, numTasks))
        else 0)

        self.writeFormatCol(predTaskId)
        self.writeFormatCol(commData)
      
      self.wf.write("\n")
      
    self.rf.close()
    self.wf.close()
  
  def readCCHP(self, readDirfile):
    self.rf = open(readDirfile, 'r')

    line = self.rf.readline()
    numTasks = self.readFormatCol(line, 0)
    numProcessors = self.readFormatCol(line, 1)

    heft = HEFT()
    heft.numTasks = numTasks + 2
    heft.createProcessors(numProcessors)

    for _ in range(numTasks+2): # +2 por causa da entry task e a exit task
      colIndex = 0
      line = self.rf.readline()
      taskId = self.readFormatCol(line, colIndex)
      colIndex += 1

      task = Task(taskId)
  
      for _ in range(numProcessors):
        cost = self.readFormatCol(line, colIndex)
        colIndex += 1
        task.compCost.append(cost)

      heft.tasks.append(task)

      numPred = self.readFormatCol(line, colIndex)
      colIndex += 1
      
      for _ in range(numPred):
        predTaskId = self.readFormatCol(line, colIndex)
        colIndex += 1
        commData = self.readFormatCol(line, colIndex)
        colIndex += 1

        edge = Edge(predTaskId, taskId, commData)

        task.predEdges.append(edge)
        heft.edges.append(edge)
        heft.numEdges += 1

    for task in heft.tasks:
      print("Task ", task.id, "-> CompCost: ", task.compCost)
      for edge in task.predEdges:
        print("Edge ", edge.predecessorTask, "->", edge.successorTask, " Data:", edge.data)
    # for edge in heft.edges:
    #   print("Edge ", edge.predecessorTask, "->", edge.successorTask, " Data:", edge.data)
    print(heft.numTasks, heft.numEdges)

    return heft


In [99]:
dag = DAG()
# dag.createCCHP('/content/drive/MyDrive/TCC/rnc50/50/rand0000 (edited).stg', '/content/drive/MyDrive/TCC/rnc50/50/rand0000 (new).stg', 2)
heft = dag.readCCHP('/content/drive/MyDrive/TCC/Grafos/topcuoglu-10.stg')

heft.calculateAvgCompCost()
heft.calculateAvgCommCost()
heft.calculateRankUpward()
for task in heft.tasks:
    print("Task ", task.id, "-> Rank: ", task.rank)
for task in heft.rankUTasks:
    print("RankUTasks ", task.id, "-> Rank: ", task.rank)
heft.scheduleTasks()
print('Schedule length = ', heft.rankUTasks[-1].duration['end'])
for p in heft.processors:  
  for t in p.taskList:
    print(p.id, t.id)


          9 0
          3 1
          0 0
          0 1
          0 2
          0 3
          0 4
          1 0
         14 1
         16 2
          9 3
          1 4
          0 5
          0 6
          2 0
         13 1
         19 2
         18 3
          1 4
          1 5
         18 6
          3 0
         11 1
         13 2
         19 3
          1 4
          1 5
         12 6
          4 0
         13 1
          8 2
         17 3
          1 4
          1 5
          9 6
          5 0
         12 1
         13 2
         10 3
          1 4
          1 5
         11 6
          6 0
         13 1
         16 2
          9 3
          1 4
          1 5
         14 6
          7 0
          7 1
         15 2
         11 3
          1 4
          3 5
         23 6
          8 0
          5 1
         11 2
         14 3
          3 4
          2 5
         19 6
          4 7
         27 8
          6 9
         15 10
          9 0
         18 1
         12 2
         20 3
     

In [100]:
# heft = HEFT()
# heft.createProcessors(2)

# heft.calculateAvgCompCost()
# heft.calculateAvgCommCost()
# heft.calculateRankUpward(heft.tasks[0])
# for task in heft.tasks:
#     print("Task ", task.id, "-> Rank: ", task.rank)
# heft.tasks.sort(key = lambda x: x.rank, reverse=True)
# for task in heft.tasks:
#     print("Task ", task.id, "-> Rank: ", task.rank)
# heft.scheduleTasks()
# for p in heft.processors:  
#   for t in p.taskList:
#     print(p.id, t.id)

In [101]:
import operator
import numpy as np
from random import randrange
# l = [0] * 10
# print(l)
# la = []
# print(np.amax(la))

# x = 1
# cost = "par" if x % 2 == 0 else "impar"
# print(cost)

# free_times = []      # no task has yet been assigned to processor
# free_times.append([0, float('inf')])
# print(free_times)

# li = [1,2,3,4]
# print(li[-1])

# class HEFT:
#   def __init__(self):
#     self.task = 2

#   def calculate(self):
#     print(self.task)

# h = HEFT()
# h.calculate()

# p, w = min(enumerate([1,2,3,2,0,3]), key=operator.itemgetter(1))
# print(p, w)

# print(enumerate([1,2,3,2,0,3]))

# cars = [1,2,3,4,5]
# for car in cars[1:]:
#   print(car)
compCost = [1, 2, 3]
commCost = compCost[:]
compCost[1] = 4
print(compCost)


[1, 4, 3]
